In [15]:
from settrade_v2 import Investor
import pandas as pd
import datetime
from cassandra.cluster import Cluster
import time

In [16]:
# 1. เชื่อมต่อ Database
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()


session.set_keyspace('stock_data')

In [17]:
# 4. Connect Settrade
investor = Investor(
    app_id="eX5XjBgNX6OoXZ14",                                 
    app_secret="ES2EcRZes9SWxnry+8jt6m3Ic+wpD/RiT8bDDp6jjvA=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)   
market = investor.MarketData() # เปลี่ยนจาก .Market() เป็น .MarketData()
print("เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!")

# 5. ทดลองดึงข้อมูล
symbol = "PTT"
candlestick = market.get_candlestick(
    symbol=symbol, 
    interval="1d", 
    limit=10
)
print(candlestick)



เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!
{'lastSequence': 440815, 'time': [1766422800, 1766509200, 1766595600, 1766682000, 1766941200, 1767027600, 1767546000, 1767632400, 1767718800, 1767805200], 'open': [32.0, 31.75, 31.75, 31.5, 31.5, 31.5, 31.75, 31.75, 31.5, 33.25], 'high': [32.25, 32.0, 31.75, 31.75, 31.75, 32.0, 32.0, 31.75, 31.75, 36.25], 'low': [31.75, 31.5, 31.5, 31.25, 31.25, 31.5, 31.5, 31.5, 31.5, 31.0], 'close': [31.75, 31.75, 31.5, 31.5, 31.5, 32.0, 31.75, 31.75, 31.5, 31.5], 'volume': [55660625, 30927077, 20536806, 34084620, 38891304, 41210512, 35819490, 41857471, 21255014, 708200], 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 22748000.0]}


In [18]:
print(dir(investor))

['Derivatives', 'Equity', 'MarketData', 'RealtimeDataConnection', 'SANDBOX', 'SANDBOX_BROKER_ID', 'SANDBOX_ENV', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_app_code', '_app_id', '_app_secret', '_broker_id', '_ctx', '_is_auto_queue']


In [20]:
import time
from datetime import datetime
import pandas as pd

# 1. ฟังก์ชันดึง candlestick
def get_candlestick_data(symbol):
    try:
        symbol = symbol.strip().upper()
        res = market.get_candlestick(
            symbol=symbol,
            interval="1d",
            limit=1000
        )
        return res
    except Exception as e:
        print(f"❌ API Error {symbol}: {e}")
        return None

# 2. ฟังก์ชัน insert ข้อมูล (เปลี่ยน ticker -> symbol ใน SQL)
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ {symbol}: ไม่มีข้อมูลในระบบ Sandbox")
        return

    time_list = res['time']
    num_rows = len(time_list)
    
    for i in range(num_rows):
        try:
            timestamp = datetime.fromtimestamp(time_list[i])
            
            # แก้ไขจาก ticker เป็น symbol ตามชื่อคอลัมน์ใหม่
            session.execute("""
                INSERT INTO candlestick_data (
                    symbol, time, open_price, high_price, low_price, close_price, volume
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,                    
                timestamp,                 
                float(res['open'][i]),     
                float(res['high'][i]),     
                float(res['low'][i]),      
                float(res['close'][i]),    
                int(res['volume'][i])      
            ))
        except Exception as e:
            print(f"❌ {symbol} บรรทัดที่ {i} ผิดพลาด: {e}")
            return 

    print(f"✅ บันทึก {symbol} สำเร็จ {num_rows} แถว")

# 3. โหลดรายชื่อหุ้นและเริ่มรัน (Limit 100)
try:
    df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/stock_pattern/data/settrade1.xlsx")
    stock_list = df['หลักทรัพย์'].dropna().unique().tolist()
    if 'หลักทรัพย์' in stock_list:
        stock_list.remove('หลักทรัพย์')
    
    # เอาแค่ 100 ตัวแรก
    stock_list = stock_list[:] 
    
    print(f"🔎 พบรายชื่อหุ้น {len(stock_list)} ")

    for symbol in stock_list:
        symbol = symbol.strip().upper()
        print(f"🚀 กำลังเริ่มดึง: {symbol}...", end=" ")
        
        res = get_candlestick_data(symbol)
        insert_historical_data(symbol, res)
        
        time.sleep(1) 

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")

🔎 พบรายชื่อหุ้น 923 
🚀 กำลังเริ่มดึง: 24CS... ❌ API Error 24CS: Symbol not found [24CS]
⚠️ 24CS: ไม่มีข้อมูลในระบบ Sandbox
🚀 กำลังเริ่มดึง: 2S... ✅ บันทึก 2S สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: 3BBIF... ✅ บันทึก 3BBIF สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: A... ✅ บันทึก A สำเร็จ 667 แถว
🚀 กำลังเริ่มดึง: A5... ✅ บันทึก A5 สำเร็จ 725 แถว
🚀 กำลังเริ่มดึง: AAI... ✅ บันทึก AAI สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: AAV... ✅ บันทึก AAV สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: ABM... ✅ บันทึก ABM สำเร็จ 724 แถว
🚀 กำลังเริ่มดึง: ACAP... ✅ บันทึก ACAP สำเร็จ 55 แถว
🚀 กำลังเริ่มดึง: ACC... ✅ บันทึก ACC สำเร็จ 728 แถว
🚀 กำลังเริ่มดึง: ACE... ✅ บันทึก ACE สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: ACG... ✅ บันทึก ACG สำเร็จ 724 แถว
🚀 กำลังเริ่มดึง: ADB... ✅ บันทึก ADB สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: ADD... ✅ บันทึก ADD สำเร็จ 728 แถว
🚀 กำลังเริ่มดึง: ADVANC... ✅ บันทึก ADVANC สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: ADVICE... ✅ บันทึก ADVICE สำเร็จ 469 แถว
🚀 กำลังเริ่มดึง: AE... ✅ บันทึก AE สำเร็จ 729 แถว
🚀 กำลังเริ่มดึง: AEONTS... ✅ บันทึก AE

KeyboardInterrupt: 

In [23]:
# ต้องใส่ ALLOW FILTERING เพราะเราไม่ได้ระบุช่วงเวลา (Clustering Key) ครบถ้วนในบางกรณี
query = "SELECT * FROM candlestick_data WHERE symbol = 'PTT' LIMIT 10 ALLOW FILTERING;"
rows = session.execute(query)
PTT = pd.DataFrame(list(rows))

print("ข้อมูลหุ้น PTT:")
print(PTT)

ข้อมูลหุ้น PTT:
  symbol       time  close_price  high_price  low_price  open_price value  \
0    PTT 2023-01-09        33.00       33.25      32.75       32.75  None   
1    PTT 2023-01-10        33.25       33.25      32.75       33.25  None   
2    PTT 2023-01-11        33.50       33.50      33.00       33.25  None   
3    PTT 2023-01-12        33.75       33.75      33.25       33.50  None   
4    PTT 2023-01-13        33.75       34.00      33.25       33.75  None   
5    PTT 2023-01-16        33.75       34.00      33.50       33.75  None   
6    PTT 2023-01-17        33.50       34.00      33.25       33.75  None   
7    PTT 2023-01-18        33.50       33.75      33.25       33.25  None   
8    PTT 2023-01-19        33.50       33.50      33.00       33.25  None   
9    PTT 2023-01-20        33.25       33.75      33.00       33.50  None   

     volume  
0  48837692  
1  39540053  
2  45435296  
3  41018316  
4  71005807  
5  67886387  
6  81916599  
7  33235123  
8  2459554